Stationarity Test:

Kwiatkowski-Phillips-Schmidt-Shin (KPSS) test

Augmented Dickey-Fuller (ADF)

ADF for the DAM

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import datetime as dt
from statsmodels.tsa.stattools import adfuller

# Load the dataset with error handling
try:
    date_format = "%d/%m/%Y %H:%M"
    date_parse = lambda date: dt.datetime.strptime(date, date_format)
    dat = pd.read_csv("/home/ciaran/Documents/DAM_VAR_1-3.csv", index_col="DeliveryPeriod", parse_dates=True, date_parser=date_parse)
    dat = dat._get_numeric_data()  # Extract numeric columns only
except Exception as e:
    print(f"Error loading data: {e}")

# Select the relevant time series for analysis
Y = dat.iloc[:, 0:1]  # Assuming the first column is EURPrices; adjust if needed

# Function to perform the ADF test
def adf_test(series):
    result = adfuller(series.dropna())  # Drop NaNs to avoid errors
    adf_statistic = result[0]
    p_value = result[1]
    critical_values = result[4]
    
    print(f'ADF Statistic: {adf_statistic}')
    print(f'p-value: {p_value}')
    for key, value in critical_values.items():
        print(f'Critical Values {key}: {value}')
    
    # Return results for potential further use
    return adf_statistic, p_value, critical_values

# Apply ADF test to the EURPrices series
print("Initial ADF Test Results:")
adf_statistic, p_value, critical_values = adf_test(Y['EURPrices'])

# Check stationarity and apply differencing if necessary
if p_value > 0.05:
    print("\nSeries is non-stationary, applying differencing...\n")
    Y_diff = Y.diff().dropna()
    
    # Reapply the ADF test to the differenced series
    print("ADF Test Results after Differencing:")
    adf_test(Y_diff['EURPrices'])
else:
    print("\nSeries is stationary, no differencing needed.")



ADF for the BM

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import datetime as dt
from statsmodels.tsa.stattools import adfuller

# Define date format for parsing
date_format = "%m/%d/%Y %H:%M"

# Function to parse dates correctly
date_parse = lambda date: dt.datetime.strptime(date, date_format)

# Load the CSV, parsing dates in the correct format and setting the index
try:
    dat = pd.read_csv("/home/ciaran/Documents/BM_data.csv", index_col="SettlementPeriod", parse_dates=True, date_parser=date_parse)
except Exception as e:
    print(f"Error loading data: {e}")

# Drop unnecessary columns and handle missing data
dat = dat.drop(["index"], axis=1, errors='ignore')  # Drop the 'index' column, if it exists
dat = dat.bfill(axis='rows')  # Backward fill missing values
dat = dat.ffill(axis='rows')  # Forward fill missing values
dat = dat._get_numeric_data()  # Extract numeric data only

# Select the relevant time series for analysis
Y = dat.iloc[:, 0:1]  # Selecting the first column; adjust as necessary

# Perform ADF test function
def adf_test(series):
    result = adfuller(series.dropna())  # Drop NaNs to avoid errors
    adf_statistic = result[0]
    p_value = result[1]
    critical_values = result[4]
    
    print(f'ADF Statistic: {adf_statistic}')
    print(f'p-value: {p_value}')
    for key, value in critical_values.items():
        print(f'Critical Values {key}: {value}')
    
    # Return the results for potential further use
    return adf_statistic, p_value, critical_values

# Apply the ADF test to the target series
print("Initial ADF Test Results:")
adf_statistic, p_value, critical_values = adf_test(Y['lag_2y'])

# Assess stationarity based on p-value and decide if differencing is needed
if p_value > 0.05:
    print("\nSeries is non-stationary, applying differencing...\n")
    Y_diff = Y.diff().dropna()  # Apply differencing if needed
    
    # Reapply the ADF test to the differenced series
    print("ADF Test Results after Differencing:")
    adf_test(Y_diff['lag_2y'])
else:
    print("\nSeries is stationary, no differencing needed.")


KPSS for the DAM

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import datetime as dt
from statsmodels.tsa.stattools import kpss

# Load the dataset
date_format = "%d/%m/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
dat = pd.read_csv("/home/ciaran/Documents/DAM_VAR_1-3.csv", index_col="DeliveryPeriod", parse_dates=True, date_parser=date_parse)

# Keep only numeric data from the dataset
dat = dat._get_numeric_data()

# Select the relevant time series for analysis (e.g., EURPrices column)
Y = dat.iloc[:, 0:1]  # Assuming EURPrices is the first column

# Define a function to perform the KPSS test
def kpss_test(series, regression='c'):
    """
    Perform KPSS test on a given time series.
    
    Parameters:
    - series: time series to test
    - regression: 'c' for constant (level stationarity), 'ct' for constant + trend (trend stationarity)
    """
    result = kpss(series.dropna(), regression=regression)
    print(f'KPSS Statistic: {result[0]}')
    print(f'p-value: {result[1]}')
    for key, value in result[3].items():
        print(f'Critical Values {key}: {value}')

    # Return the p-value to determine if differencing is required
    return result[1]

# Apply KPSS test to check for level stationarity (regression='c')
print("KPSS Test for Level Stationarity:")
p_value = kpss_test(Y['EURPrices'], regression='c')

# If the series is not stationary (p-value < 0.05), apply differencing
if p_value < 0.05:
    print("\nSeries is not stationary. Applying differencing...")
    Y_diff = Y.diff().dropna()
    
    # Reapply the KPSS test to the differenced data
    print("KPSS Test on Differenced Data:")
    kpss_test(Y_diff['EURPrices'], regression='c')
else:
    print("\nSeries is stationary. No differencing required.")


KPSS test for the BM

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import datetime as dt
from statsmodels.tsa.stattools import kpss

# Define date format for parsing
date_format = "%m/%d/%Y %H:%M"

# Function to parse dates
date_parse = lambda date: dt.datetime.strptime(date, date_format)

# Read the CSV file and parse dates
dat = pd.read_csv("/home/ciaran/Documents/BM_data.csv", index_col="SettlementPeriod", parse_dates=True, date_parser=date_parse)

# Remove any non-numeric columns if necessary (like 'index')
dat = dat.drop(["index"], axis=1)

# Convert to DataFrame and handle missing values with forward and backward fill
dat = pd.DataFrame(dat)
dat = dat.bfill(axis='rows')
dat = dat.ffill(axis='rows')

# Keep only numeric data
dat = dat._get_numeric_data()

# Select the target time series for analysis (lag_2y)
Y = dat[['lag_2y']]

# Define a function to perform the KPSS test
def kpss_test(series, regression='c'):
    """
    Perform KPSS test on a given time series.
    
    Parameters:
    - series: time series to test
    - regression: 'c' for constant (level stationarity), 'ct' for constant + trend (trend stationarity)
    """
    result = kpss(series.dropna(), regression=regression)
    print(f'KPSS Statistic: {result[0]}')
    print(f'p-value: {result[1]}')
    for key, value in result[3].items():
        print(f'Critical Values {key}: {value}')

    # Return the p-value to determine if differencing is required
    return result[1]

# Apply KPSS test to check for level stationarity (regression='c')
print("KPSS Test for Level Stationarity (lag_2y):")
p_value = kpss_test(Y['lag_2y'], regression='c')

# If the series is not stationary (p-value < 0.05), apply differencing
if p_value < 0.05:
    print("\nSeries is not stationary. Applying differencing...")
    Y_diff = Y.diff().dropna()
    
    # Reapply the KPSS test to the differenced data
    print("KPSS Test on Differenced Data (lag_2y):")
    kpss_test(Y_diff['lag_2y'], regression='c')
else:
    print("\nSeries is stationary. No differencing required.")


Autocorrelation and Partial Autocorrelation Analysis:

ACF and PACF


ACF for the DAM

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import datetime as dt

# Step 1: Load the dataset
date_format = "%d/%m/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
dat = pd.read_csv("/home/ciaran/Documents/DAM_VAR_1-3.csv", index_col="DeliveryPeriod", parse_dates=True, date_format=date_format)
dat = dat._get_numeric_data()

# Step 2: Extract the time series data for DAM prices and other covariates
Y_dam = dat['EURPrices']
covariates = ['DF', 'WF']

# Step 3: Plot ACF for the DAM data and covariates
def plot_acf(series, lags=167, ax=None):
    """
    Plot the Autocorrelation Function (ACF) for a given time series.

    Parameters:
    - series: The time series data to plot.
    - lags: Number of lags to include in the plot.
    - ax: Matplotlib axis object for plotting.
    """
    sm.graphics.tsa.plot_acf(series, lags=lags, ax=ax, color='blue')
    ax.set_xlabel('Lag', fontsize=12)
    ax.set_ylabel('ACF', fontsize=12)
    ax.tick_params(axis='both', which='major', labelsize=10)
    ax.grid(True, linestyle='--', alpha=0.7)
    ax.set_title(f'ACF - {series.name}', fontsize=14, fontweight='bold')


# Define figure and axes
fig, axes = plt.subplots(1, 3, figsize=(18, 6), sharey=True)
fig.suptitle('Autocorrelation Function Analysis for the Day-Ahead Market Dataset', fontsize=16, fontweight='bold')

# Plot ACF for EURPrices
plot_acf(Y_dam, lags=167, ax=axes[0])
axes[0].set_title('Day-Ahead Market Prices', fontsize=14, fontweight='bold')

# Plot ACF for DF
if 'DF' in dat.columns:
    plot_acf(dat['DF'], lags=167, ax=axes[1])
    axes[1].set_title('Demand Forecast', fontsize=14, fontweight='bold')
else:
    axes[1].set_visible(False)  # Hide subplot if 'DF' column is missing

# Plot ACF for WF
if 'WF' in dat.columns:
    plot_acf(dat['WF'], lags=167, ax=axes[2])
    axes[2].set_title('Wind Forecast', fontsize=14, fontweight='bold')
else:
    axes[2].set_visible(False)  # Hide subplot if 'WF' column is missing

# Adjust layout and display
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.savefig('/home/ciaran/ACF_DAM.jpg', format='jpg', dpi=300)

# Display the plot
plt.show()

ACF for the BM

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import datetime as dt

# Define date format for parsing
date_format = "%m/%d/%Y %H:%M"

# Function to parse dates
date_parse = lambda date: dt.datetime.strptime(date, date_format)

# Read CSV without frequency information and parse dates
dat = pd.read_csv("/home/ciaran/Documents/BM_data.csv", index_col="SettlementPeriod", parse_dates=True, date_parser=date_parse)

# Drop non-numeric columns and handle missing data
dat = dat.drop(["index"], axis=1)
dat = dat.bfill(axis='rows').ffill(axis='rows')._get_numeric_data()

# Select the relevant time series for analysis
Y = dat.iloc[:, 0:1]  # EURPrices column
X = dat.iloc[:, 16:]  # Other variables
X = pd.concat([
    X.iloc[:, :1],     # lag_-3x1
    X.iloc[:, 49:50],  # lag_-3x2
    X.iloc[:, 98:99],  # lag_-2x3
    X.iloc[:, 147:148],# lag_0x4
    X.iloc[:, 148:149],# lag_0x5
    X.iloc[:, 149:150],# lag_0x6
    X.iloc[:, 197:198],# lag_2x7
    X.iloc[:, 213:214],# lag_2x8
    X.iloc[:, 229:230],# lag_2x9
    X.iloc[:, 245:246],# lag_2x10
    X.iloc[:, 261:262],# lag_2x11
    X.iloc[:, 277:278] # lag_-2x12
], axis=1)

# Extract specific series for historical and future-looking data
historical_data = {
    'BMP': X['lag_-3x1'],
    'BMV': X['lag_-3x2'],
    'WDiff': X['lag_-2x3'],
    'I': X['lag_-2x12'],
    'DAM': X['lag_0x6']
}

future_data = {
    'PHPN': X['lag_2x7'],
    'PHI': X['lag_2x8'],
    'PHFW': X['lag_2x9'],
    'PHFD': X['lag_2x10'],
    'DAM': X['lag_2x11']
}

# Function to plot ACF
def plot_acf(series, lags=48, ax=None):
    """
    Plot the Autocorrelation Function (ACF) for a given time series.

    Parameters:
    - series: The time series data to plot.
    - lags: Number of lags to include in the plot.
    - ax: Matplotlib axis object for plotting.
    """
    sm.graphics.tsa.plot_acf(series, lags=lags, ax=ax, color='blue')
    ax.set_xlabel('Lag', fontsize=12)
    ax.set_ylabel('ACF', fontsize=12)
    ax.tick_params(axis='both', which='major', labelsize=10)
    ax.grid(True, linestyle='--', alpha=0.7)
    ax.set_title(f'ACF - {series.name}', fontsize=14, fontweight='bold')

# Create subplots for historical and future-looking data
fig, axes = plt.subplots(2, 4, figsize=(36, 12), sharey=True)
fig.suptitle('Autocorrelation Function Analysis for Balancing Market Dataset', fontsize=16, fontweight='bold')

# Plot ACF for historical data (first row)
plot_acf(historical_data['BMP'], lags=48, ax=axes[0, 0])
axes[0, 0].set_title('BM Price', fontsize=14, fontweight='bold')

plot_acf(historical_data['BMV'], lags=48, ax=axes[0, 1])
axes[0, 1].set_title('BM Volume', fontsize=14, fontweight='bold')

plot_acf(historical_data['WDiff'], lags=48, ax=axes[0, 2])
axes[0, 2].set_title('Forecast Wind - Actual Wind:', fontsize=16, fontweight='bold')

plot_acf(historical_data['I'], lags=48, ax=axes[0, 3])
axes[0, 3].set_title('Interconnector Values', fontsize=14, fontweight='bold')

# plot_acf(historical_data['DAM'], lags=48, ax=axes[0, 4])
# axes[0, 4].set_title('Past DAM Prices', fontsize=14, fontweight='bold')

# Plot ACF for future-looking data (second row)
plot_acf(future_data['PHPN'], lags=48, ax=axes[1, 0])
axes[1, 0].set_title('Physical Notifications Volume', fontsize=14, fontweight='bold')

plot_acf(future_data['PHI'], lags=48, ax=axes[1, 1])
axes[1, 1].set_title('Net Interconnector Schedule', fontsize=14, fontweight='bold')

plot_acf(future_data['PHFW'], lags=48, ax=axes[1, 2])
axes[1, 2].set_title('Renewable Forecast', fontsize=14, fontweight='bold')

plot_acf(future_data['PHFD'], lags=48, ax=axes[1, 3])
axes[1, 3].set_title('Demand Forecast', fontsize=14, fontweight='bold')

# DAM is included twice in both historical and future data; use a distinct title for clarity
# plot_acf(future_data['DAM'], lags=48, ax=axes[1, 4])
# axes[1, 4].set_title('Future DAM Prices', fontsize=14, fontweight='bold')

# Adjust layout and save the figure
plt.tight_layout(rect=[0, 0, 1, 0.95])

# Save the figure as a JPG file
plt.savefig('/home/ciaran/BM_ACF.jpg', format='jpg', dpi=500)

# Display the plot
plt.show()


Partial Autocorrelation Function for the DAM

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import datetime as dt

# Step 1: Load the dataset
date_format = "%d/%m/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
dat = pd.read_csv("/home/ciaran/Documents/DAM_VAR_1-3.csv", index_col="DeliveryPeriod", parse_dates=True, date_parser=date_parse)
dat = dat._get_numeric_data()

# Step 2: Extract the time series data for DAM prices and other covariates
Y_dam = dat['EURPrices']
covariates = ['DF', 'WF']

# Step 3: Plot PACF for the DAM data and covariates
def plot_pacf(series, lags=48, ax=None):
    """
    Plot the Partial Autocorrelation Function (PACF) for a given time series.

    Parameters:
    - series: The time series data to plot.
    - lags: Number of lags to include in the plot.
    - ax: Matplotlib axis object for plotting.
    """
    sm.graphics.tsa.plot_pacf(series, lags=lags, ax=ax, method='ywm', color='blue')
    ax.set_xlabel('Lag', fontsize=12)
    ax.set_ylabel('PACF', fontsize=12)
    ax.tick_params(axis='both', which='major', labelsize=10)
    ax.grid(True, linestyle='--', alpha=0.7)
    ax.set_title(f'PACF - {series.name}', fontsize=14, fontweight='bold')

# Define figure and axes
fig, axes = plt.subplots(1, 3, figsize=(18, 6), sharey=True)
fig.suptitle('Partial Autocorrelation Function Analysis for the Day-Ahead Market Dataset', fontsize=16, fontweight='bold')

# Plot PACF for EURPrices
plot_pacf(Y_dam, lags=167, ax=axes[0])
axes[0].set_title('Day-Ahead Market Prices', fontsize=14, fontweight='bold')

# Plot PACF for DF
if 'DF' in dat.columns:
    plot_pacf(dat['DF'], lags=167, ax=axes[1])
    axes[1].set_title('Demand Forecast', fontsize=14, fontweight='bold')
else:
    axes[1].set_visible(False)  # Hide subplot if 'DF' column is missing

# Plot PACF for WF
if 'WF' in dat.columns:
    plot_pacf(dat['WF'], lags=167, ax=axes[2])
    axes[2].set_title('Wind Forecast', fontsize=14, fontweight='bold')
else:
    axes[2].set_visible(False)  # Hide subplot if 'WF' column is missing

# Adjust layout and display
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.savefig('/home/ciaran/PACF_DAM.jpg', format='jpg', dpi=300)

# Display the plot
plt.show()


Partial Autocorrelation Function for the BM

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import datetime as dt

# Define date format for parsing
date_format = "%m/%d/%Y %H:%M"

# Function to parse dates
date_parse = lambda date: dt.datetime.strptime(date, date_format)

# Read CSV without frequency information and parse dates
dat = pd.read_csv("/home/ciaran/Documents/BM_data.csv", index_col="SettlementPeriod", parse_dates=True, date_parser=date_parse)

# Drop non-numeric columns and handle missing data
dat = dat.drop(["index"], axis=1)
dat = dat.bfill(axis='rows').ffill(axis='rows')._get_numeric_data()

# Select the relevant time series for analysis
Y = dat.iloc[:, 0:1]  # EURPrices column
X = dat.iloc[:, 16:]  # Other variables
X = pd.concat([
    X.iloc[:, :1],     # lag_-3x1
    X.iloc[:, 49:50],  # lag_-3x2
    X.iloc[:, 98:99],  # lag_-2x3
    X.iloc[:, 147:148],# lag_0x4
    X.iloc[:, 148:149],# lag_0x5
    X.iloc[:, 149:150],# lag_0x6
    X.iloc[:, 197:198],# lag_2x7
    X.iloc[:, 213:214],# lag_2x8
    X.iloc[:, 229:230],# lag_2x9
    X.iloc[:, 245:246],# lag_2x10
    X.iloc[:, 261:262],# lag_2x11
    X.iloc[:, 277:278] # lag_-2x12
], axis=1)

# Extract specific series for historical and future-looking data
historical_data = {
    'BMP': X['lag_-3x1'],
    'BMV': X['lag_-3x2'],
    'WDiff': X['lag_-2x3'],
    'I': X['lag_-2x12'],
    'DAM': X['lag_0x6']
}

future_data = {
    'PHPN': X['lag_2x7'],
    'PHI': X['lag_2x8'],
    'PHFW': X['lag_2x9'],
    'PHFD': X['lag_2x10'],
    'DAM': X['lag_2x11']
}

# Function to plot PACF
def plot_pacf(series, lags=48, ax=None):
    """
    Plot the Partial Autocorrelation Function (PACF) for a given time series.

    Parameters:
    - series: The time series data to plot.
    - lags: Number of lags to include in the plot.
    - ax: Matplotlib axis object for plotting.
    """
    sm.graphics.tsa.plot_pacf(series, lags=lags, ax=ax, method='ywm', color='blue')
    ax.set_xlabel('Lag', fontsize=12)
    ax.set_ylabel('PACF', fontsize=12)
    ax.tick_params(axis='both', which='major', labelsize=10)
    ax.grid(True, linestyle='--', alpha=0.7)
    ax.set_title(f'PACF - {series.name}', fontsize=14, fontweight='bold')

# Create subplots for historical and future-looking data
fig, axes = plt.subplots(2, 4, figsize=(36, 12), sharey=True)
fig.suptitle('Partial Autocorrelation Function Analysis for Balancing Market Dataset', fontsize=16, fontweight='bold')

# Plot PACF for historical data (first row)
plot_pacf(historical_data['BMP'], lags=48, ax=axes[0, 0])
axes[0, 0].set_title('BM Prices', fontsize=14, fontweight='bold')

plot_pacf(historical_data['BMV'], lags=48, ax=axes[0, 1])
axes[0, 1].set_title('BM Volume', fontsize=14, fontweight='bold')

plot_pacf(historical_data['WDiff'], lags=48, ax=axes[0, 2])
axes[0, 2].set_title('Forecast Wind - Actual Wind', fontsize=14, fontweight='bold')

plot_pacf(historical_data['I'], lags=48, ax=axes[0, 3])
axes[0, 3].set_title('Interconnector Values', fontsize=14, fontweight='bold')

# plot_pacf(historical_data['DAM'], lags=48, ax=axes[0, 4])
# axes[0, 4].set_title('DAM', fontsize=14, fontweight='bold')

# Plot PACF for future-looking data (second row)
plot_pacf(future_data['PHPN'], lags=48, ax=axes[1, 0])
axes[1, 0].set_title('Physical Notifications Volume', fontsize=14, fontweight='bold')

plot_pacf(future_data['PHI'], lags=48, ax=axes[1, 1])
axes[1, 1].set_title('Net Interconnector Schedule', fontsize=14, fontweight='bold')

plot_pacf(future_data['PHFW'], lags=48, ax=axes[1, 2])
axes[1, 2].set_title('Renewable Forecast', fontsize=14, fontweight='bold')

plot_pacf(future_data['PHFD'], lags=48, ax=axes[1, 3])
axes[1, 3].set_title('Demand Forecast', fontsize=14, fontweight='bold')

# DAM is included twice in both historical and future data; use a distinct title for clarity
# plot_pacf(future_data['DAM'], lags=48, ax=axes[1, 4])
# axes[1, 4].set_title('DAM (Future)', fontsize=14, fontweight='bold')

# Adjust layout and save the figure
plt.tight_layout(rect=[0, 0, 1, 0.95])

# Save the figure as a JPG file
plt.savefig('/home/ciaran/BM_PACF.jpg', format='jpg', dpi=500)

# Display the plot
plt.show()



Serial Correlation Tests:

1.) Ljung-Box Test

2.) Breusch-Godfrey Test


Ljung-Box Test for the Day-Ahead Market

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
import statsmodels.api as sm
from statsmodels.stats.diagnostic import acorr_breusch_godfrey

# Define the date format for parsing
date_format = "%d/%m/%Y %H:%M"
date_parse = lambda date: pd.to_datetime(date, format=date_format)

# Load the dataset
dat = pd.read_csv("/home/ciaran/Documents/DAM_VAR_1-3.csv", index_col="DeliveryPeriod", parse_dates=True, date_parser=date_parse)
dat = dat._get_numeric_data()

Y = dat.iloc[:, 0:1]  # The first column as the dependent variable
X = dat.iloc[:, 24:]  # Select all independent variables from the 24th column onward
X = pd.concat([X.iloc[:, :1], X.iloc[:, 144:145], X.iloc[:, 168:169], X.iloc[:, 312:313], X.iloc[:, 336:337]], axis=1)  # Select specific columns

# Handling missing values by filling with the mean
X = X.fillna(X.mean())

# Splitting data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# Fit Lasso model
lasso = Lasso(alpha=0.01)  # Regularization parameter
lasso.fit(X_train, Y_train)

# Predict and calculate residuals for Lasso
Y_pred_lasso = lasso.predict(X_test)
Y_pred_lasso = pd.Series(Y_pred_lasso, index=Y_test.index)  # Convert predictions to a Series with the same index as Y_test
residuals_lasso = Y_test.iloc[:, 0] - Y_pred_lasso  # Ensure both are Series with the same length

# Perform Ljung-Box test on Lasso residuals
lags = 24  # Number of lags for the test
ljung_box_result_lasso = sm.stats.acorr_ljungbox(residuals_lasso, lags=lags, return_df=True)

# Select the maximum lag's statistic and p-value as a single summary value
max_lag_stat = ljung_box_result_lasso.iloc[-1]  # Taking the last row for lag 24

print("Ljung-Box Test Summary for Lasso Model at Maximum:")
print(f"Ljung-Box Statistic Lag (24): {max_lag_stat['lb_stat']:.4f}, p-value: {max_lag_stat['lb_pvalue']:.4f}")

# Perform Ljung-Box test on Lasso residuals
lags = 48  # Number of lags for the test
ljung_box_result_lasso = sm.stats.acorr_ljungbox(residuals_lasso, lags=lags, return_df=True)

# Select the maximum lag's statistic and p-value as a single summary value
max_lag_stat = ljung_box_result_lasso.iloc[-1]  # Taking the last row for lag 24

print("Ljung-Box Test Summary for Lasso Model at Maximum:")
print(f"Ljung-Box Statistic Lag (48): {max_lag_stat['lb_stat']:.4f}, p-value: {max_lag_stat['lb_pvalue']:.4f}")

# Perform Ljung-Box test on Lasso residuals
lags = 96  # Number of lags for the test
ljung_box_result_lasso = sm.stats.acorr_ljungbox(residuals_lasso, lags=lags, return_df=True)

# Select the maximum lag's statistic and p-value as a single summary value
max_lag_stat = ljung_box_result_lasso.iloc[-1]  # Taking the last row for lag 24

print("Ljung-Box Test Summary for Lasso Model at Maximum:")
print(f"Ljung-Box Statistic Lag (96): {max_lag_stat['lb_stat']:.4f}, p-value: {max_lag_stat['lb_pvalue']:.4f}")


# Perform Ljung-Box test on Lasso residuals
lags = 167  # Number of lags for the test
ljung_box_result_lasso = sm.stats.acorr_ljungbox(residuals_lasso, lags=lags, return_df=True)

# Select the maximum lag's statistic and p-value as a single summary value
max_lag_stat = ljung_box_result_lasso.iloc[-1]  # Taking the last row for lag 24

print("Ljung-Box Test Summary for Lasso Model at Maximum:")
print(f"Ljung-Box Statistic Lag (167): {max_lag_stat['lb_stat']:.4f}, p-value: {max_lag_stat['lb_pvalue']:.4f}")


Breusch-Godfrey test Day-Ahead Market

In [ ]:
lags=24
ols_model = sm.OLS(Y_train, sm.add_constant(X_train)).fit()
bg_test = acorr_breusch_godfrey(ols_model, nlags=lags)
bg_stat, bg_pvalue, _, _ = bg_test
print("\nBreusch-Godfrey Test Results for Lasso Model Residuals:")
print(f"Breusch-Godfrey Statistic lags 24: {bg_stat:.4f}, p-value: {bg_pvalue:.4f}")

lags=48
ols_model = sm.OLS(Y_train, sm.add_constant(X_train)).fit()
bg_test = acorr_breusch_godfrey(ols_model, nlags=lags)
bg_stat, bg_pvalue, _, _ = bg_test
print("\nBreusch-Godfrey Test Results for Lasso Model Residuals:")
print(f"Breusch-Godfrey Statistic lags 48: {bg_stat:.4f}, p-value: {bg_pvalue:.4f}")

lags=96
ols_model = sm.OLS(Y_train, sm.add_constant(X_train)).fit()
bg_test = acorr_breusch_godfrey(ols_model, nlags=lags)
bg_stat, bg_pvalue, _, _ = bg_test
print("\nBreusch-Godfrey Test Results for Lasso Model Residuals:")
print(f"Breusch-Godfrey Statistic lags 96: {bg_stat:.4f}, p-value: {bg_pvalue:.4f}")

lags=167
ols_model = sm.OLS(Y_train, sm.add_constant(X_train)).fit()
bg_test = acorr_breusch_godfrey(ols_model, nlags=lags)
bg_stat, bg_pvalue, _, _ = bg_test
print("\nBreusch-Godfrey Test Results for Lasso Model Residuals :")
print(f"Breusch-Godfrey Statistic lags 167: {bg_stat:.4f}, p-value: {bg_pvalue:.4f}")

Ljung-Box Test for the Balancing Market

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
import statsmodels.api as sm
from statsmodels.stats.diagnostic import acorr_breusch_godfrey

# Define date format for parsing
date_format = "%m/%d/%Y %H:%M"

# Function to parse dates
date_format = lambda date: dt.datetime.strptime(date, date_format)

# Read CSV without frequency information and parse dates
dat = pd.read_csv("/home/ciaran/Documents/BM_data.csv", index_col="SettlementPeriod", parse_dates=True, date_format=date_format)

# Specify targets for forecasting
dat = dat.drop(["index"], axis=1)
dat = pd.DataFrame(dat)
dat = dat.bfill(axis='rows')
dat = dat.ffill(axis='rows')
dat = dat._get_numeric_data()

# Select the relevant time series for analysis
Y = dat.iloc[:, 0:1]  # EURPrices column
X = dat.iloc[:, 16:]  # Other variables
X = pd.concat([X.iloc[:, :1], X.iloc[:, 49:50], X.iloc[:, 98:99], X.iloc[:, 147:148], X.iloc[:, 148:149],
X.iloc[:, 149:150], X.iloc[:, 197:198], X.iloc[:, 213:214], X.iloc[:, 229:230], 
X.iloc[:, 245:246], X.iloc[:, 261:262], X.iloc[:, 277:278]], axis=1)

# Handling missing values by filling with the mean
X = X.fillna(X.mean())

# Splitting data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# Fit Lasso model
lasso = Lasso(alpha=0.01)  # Regularization parameter
lasso.fit(X_train, Y_train)

# Predict and calculate residuals for Lasso
Y_pred_lasso = lasso.predict(X_test)
Y_pred_lasso = pd.Series(Y_pred_lasso, index=Y_test.index)  # Convert predictions to a Series with the same index as Y_test
residuals_lasso = Y_test.iloc[:, 0] - Y_pred_lasso  # Ensure both are Series with the same length

lags = 16  # Number of lags for the test
ljung_box_result_lasso = sm.stats.acorr_ljungbox(residuals_lasso, lags=lags, return_df=True)
max_lag_stat = ljung_box_result_lasso.iloc[-1]  # Taking the last row for lag 24
print("Ljung-Box Test Summary for Lasso Model at Maximum:")
print(f"Ljung-Box Statistic Lag (16): {max_lag_stat['lb_stat']:.4f}, p-value: {max_lag_stat['lb_pvalue']:.4f}")

lags = 32  # Number of lags for the test
ljung_box_result_lasso = sm.stats.acorr_ljungbox(residuals_lasso, lags=lags, return_df=True)
max_lag_stat = ljung_box_result_lasso.iloc[-1]  # Taking the last row for lag 24
print("Ljung-Box Test Summary for Lasso Model at Maximum:")
print(f"Ljung-Box Statistic Lag (32): {max_lag_stat['lb_stat']:.4f}, p-value: {max_lag_stat['lb_pvalue']:.4f}")

lags = 48  # Number of lags for the test
ljung_box_result_lasso = sm.stats.acorr_ljungbox(residuals_lasso, lags=lags, return_df=True)
max_lag_stat = ljung_box_result_lasso.iloc[-1]  # Taking the last row for lag 24
print("Ljung-Box Test Summary for Lasso Model at Maximum:")
print(f"Ljung-Box Statistic Lag (48): {max_lag_stat['lb_stat']:.4f}, p-value: {max_lag_stat['lb_pvalue']:.4f}")

lags = 96  # Number of lags for the test
ljung_box_result_lasso = sm.stats.acorr_ljungbox(residuals_lasso, lags=lags, return_df=True)
max_lag_stat = ljung_box_result_lasso.iloc[-1]  # Taking the last row for lag 24
print("Ljung-Box Test Summary for Lasso Model at Maximum:")
print(f"Ljung-Box Statistic Lag (96): {max_lag_stat['lb_stat']:.4f}, p-value: {max_lag_stat['lb_pvalue']:.4f}")

Breusch-Godfrey test Balancing Market

In [ ]:
lags=16
ols_model = sm.OLS(Y_train, sm.add_constant(X_train)).fit()
bg_test = acorr_breusch_godfrey(ols_model, nlags=lags)
bg_stat, bg_pvalue, _, _ = bg_test
print("\nBreusch-Godfrey Test Results for Lasso Model Residuals:")
print(f"Breusch-Godfrey Statistic Lag (16): {bg_stat:.4f}, p-value: {bg_pvalue:.4f}")

lags=32
ols_model = sm.OLS(Y_train, sm.add_constant(X_train)).fit()
bg_test = acorr_breusch_godfrey(ols_model, nlags=lags)
bg_stat, bg_pvalue, _, _ = bg_test
print("\nBreusch-Godfrey Test Results for Lasso Model Residuals:")
print(f"Breusch-Godfrey Statistic Lag (32): {bg_stat:.4f}, p-value: {bg_pvalue:.4f}")

lags=48
ols_model = sm.OLS(Y_train, sm.add_constant(X_train)).fit()
bg_test = acorr_breusch_godfrey(ols_model, nlags=lags)
bg_stat, bg_pvalue, _, _ = bg_test
print("\nBreusch-Godfrey Test Results for Lasso Model Residuals:")
print(f"Breusch-Godfrey Statistic Lag (48): {bg_stat:.4f}, p-value: {bg_pvalue:.4f}")

lags=96
ols_model = sm.OLS(Y_train, sm.add_constant(X_train)).fit()
bg_test = acorr_breusch_godfrey(ols_model, nlags=lags)
bg_stat, bg_pvalue, _, _ = bg_test
print("\nBreusch-Godfrey Test Results for Lasso Model Residuals:")
print(f"Breusch-Godfrey Statistic Lag (96): {bg_stat:.4f}, p-value: {bg_pvalue:.4f}")

Heteroscedasticity Tests-

Breusch-Pagan Test

White Test


Breusch-Pagan Test for the Day-Ahead Market

In [ ]:
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_breuschpagan

# Load the data
date_format = "%d/%m/%Y %H:%M"
dat = pd.read_csv("/home/ciaran/Documents/DAM_VAR_1-3.csv", index_col="DeliveryPeriod", parse_dates=True, date_format=date_format)
dat = dat._get_numeric_data()

# Define the dependent (Y) and independent (X) variables
Y = dat.iloc[:, 0]  # Dependent variable (first column)
X = dat.iloc[:, 24:]  # Independent variables starting from the 24th column

# Selecting specific columns from the independent variables
X = pd.concat([X.iloc[:, :1], X.iloc[:, 144:145], X.iloc[:, 168:169], X.iloc[:, 312:313], X.iloc[:, 336:337]], axis=1)

# Split data into training and testing sets (80% train, 20% test)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

# Standardize the features (Lasso is sensitive to scale)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Fit Lasso regression with a specific alpha value (regularization strength)
lasso_model = Lasso(alpha=0.1)
lasso_model.fit(X_train_scaled, Y_train)  # No need to scale Y_train, model will predict on the same scale

# Calculate residuals on the TRAINING set
Y_train_pred = lasso_model.predict(X_train_scaled)
residuals_train = Y_train - Y_train_pred  # Correct because Y_train was not scaled

# Perform the Breusch-Pagan Test
# Add a constant to the independent variables (required for the test)
X_train_with_const = sm.add_constant(X_train_scaled)

# Perform the Breusch-Pagan test on the training set residuals
bp_test = het_breuschpagan(residuals_train, X_train_with_const)

# Extract the test results
bp_stat, bp_pvalue, fvalue, f_pvalue = bp_test

# Print the Breusch-Pagan Test results
print(f"Breusch-Pagan Test Statistic (LM): {bp_stat}")
print(f"p-value (LM test): {bp_pvalue}")
print(f"F-statistic: {fvalue}")
print(f"p-value (F-test): {f_pvalue}")


Breusch-Pagan Test for the Balancing Market

In [ ]:
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_breuschpagan

# Load the data
date_format = "%m/%d/%Y %H:%M"
date_format = lambda date: dt.datetime.strptime(date, date_format)
dat = pd.read_csv("/home/ciaran/Documents/BM_data.csv", index_col="SettlementPeriod", parse_dates=True, date_format=date_format)
dat = dat.drop(["index"], axis=1)
dat = pd.DataFrame(dat)
dat = dat.bfill(axis='rows')
dat = dat.ffill(axis='rows')
dat = dat._get_numeric_data()

# Define dependent (Y) and independent (X) variables
Y = dat.iloc[:, 0:1]  # EURPrices column
X = dat.iloc[:, 16:]  # Other variables
X = pd.concat([X.iloc[:, :1], X.iloc[:, 49:50], X.iloc[:, 98:99], X.iloc[:, 147:148], X.iloc[:, 148:149],
               X.iloc[:, 149:150], X.iloc[:, 197:198], X.iloc[:, 213:214], X.iloc[:, 229:230], 
               X.iloc[:, 245:246], X.iloc[:, 261:262], X.iloc[:, 277:278]], axis=1)

# Split data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

# Standardize the features (Lasso is sensitive to scale)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Fit Lasso regression
lasso_model = Lasso(alpha=0.1)
lasso_model.fit(X_train_scaled, Y_train)

# Calculate residuals on the training set
Y_train_pred = lasso_model.predict(X_train_scaled)

# Convert Y_train to a 1D array for compatibility with Y_train_pred
Y_train_flat = Y_train.values.ravel()  # Flatten Y_train

# Calculate residuals
residuals_train = Y_train_flat - Y_train_pred

# Perform the Breusch-Pagan Test
# Add a constant to the independent variables (required for the test)
X_train_with_const = sm.add_constant(X_train_scaled)

# Perform the Breusch-Pagan test on the training set residuals
bp_test = het_breuschpagan(residuals_train, X_train_with_const)

# Extract the test results
bp_stat, bp_pvalue, fvalue, f_pvalue = bp_test

# Print the Breusch-Pagan Test results
print(f"Breusch-Pagan Test Statistic (LM): {bp_stat}")
print(f"p-value (LM test): {bp_pvalue}")
print(f"F-statistic: {fvalue}")
print(f"p-value (F-test): {f_pvalue}")


White Test for the Day-Ahead Market

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from itertools import combinations
from scipy import stats

# Step 1: Load the data
date_format = "%d/%m/%Y %H:%M"
dat = pd.read_csv("/home/ciaran/Documents/DAM_VAR_1-3.csv", index_col="DeliveryPeriod", parse_dates=True, date_format=date_format)
dat = dat._get_numeric_data()

# Step 2: Define the dependent (Y) and independent (X) variables
Y = dat.iloc[:, 0]  # Dependent variable (first column)
X = dat.iloc[:, 24:]  # Independent variables starting from the 24th column

# Step 3: Selecting specific columns from the independent variables
X = pd.concat([X.iloc[:, :1], X.iloc[:, 144:145], X.iloc[:, 168:169], X.iloc[:, 312:313], X.iloc[:, 336:337]], axis=1)

# Step 4: Split data into training and testing sets (80% train, 20% test)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Step 5: Standardize the features (Lasso is sensitive to scale)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert the scaled X_train back to a DataFrame to keep the index alignment with Y_train
X_train_scaled = pd.DataFrame(X_train_scaled, index=X_train.index, columns=X_train.columns)

# Step 6: Fit Lasso regression with a specific alpha value (regularization strength)
lasso_model = Lasso(alpha=0.1)
lasso_model.fit(X_train_scaled, Y_train)

# Step 7: Calculate residuals on the TRAINING set
Y_train_pred = lasso_model.predict(X_train_scaled)
residuals_train = Y_train - Y_train_pred

# Step 8: Create auxiliary regression dataset with squared terms and interaction terms
def create_white_test_data(X):
    X_df = pd.DataFrame(X)  # Input data already has a constant added if necessary
    
    # Add squared terms
    squared_terms = {f'{col}^2': X_df[col] ** 2 for col in X_df.columns}
    
    # Add interaction terms
    interaction_terms = {f'{col1}*{col2}': X_df[col1] * X_df[col2] 
                         for col1, col2 in combinations(X_df.columns, 2)}
    
    # Combine original, squared, and interaction terms into a single DataFrame
    X_expanded = pd.concat([X_df, pd.DataFrame(squared_terms), pd.DataFrame(interaction_terms)], axis=1)

    return X_expanded

# Step 9: Run the adapted White Test
def run_adapted_white_test(X, residuals):
    # Create the expanded auxiliary regression dataset
    X_auxiliary = create_white_test_data(X)
    
    # Fit the auxiliary regression with squared residuals as the target
    auxiliary_model = sm.OLS(residuals**2, sm.add_constant(X_auxiliary)).fit()
    
    # Calculate the White test statistic
    n = X_auxiliary.shape[0]  # Number of observations
    R_squared = auxiliary_model.rsquared
    white_statistic = n * R_squared  # White test statistic (LM test)

    # Degrees of freedom for chi-square test
    df = X_auxiliary.shape[1]  # Degrees of freedom for the test statistic
    
    # p-value from chi-square distribution using scipy
    p_value = stats.chi2.sf(white_statistic, df)
    
    return white_statistic, p_value, auxiliary_model

# Step 10: Apply the adapted White Test on the training data
white_stat, p_value, auxiliary_model = run_adapted_white_test(X_train_scaled, residuals_train)

# Step 11: Print the results
print(f"White Test Statistic: {white_stat}")
print(f"p-value: {p_value}")


White Test for the Balancing Market

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from itertools import combinations
from scipy import stats

# Step 1: Load the data
date_format = "%m/%d/%Y %H:%M"
dat = pd.read_csv("/home/ciaran/Documents/BM_data.csv", index_col="SettlementPeriod", parse_dates=True)
dat = dat.drop(["index"], axis=1, errors='ignore')  # Drop 'index' column if it exists
dat = dat.bfill(axis='rows').ffill(axis='rows')  # Backfill and forward fill missing values
dat = dat._get_numeric_data()  # Keep only numeric data

# Step 2: Define dependent (Y) and independent (X) variables
Y = dat.iloc[:, 0]  # Assuming the first column represents the dependent variable (e.g., EURPrices)
X = dat.iloc[:, 16:]  # Starting from the 17th column as independent variables

# Step 3: Selecting specific columns from the independent variables
X = pd.concat([X.iloc[:, :1], X.iloc[:, 49:50], X.iloc[:, 98:99], X.iloc[:, 147:148], X.iloc[:, 148:149],
               X.iloc[:, 149:150], X.iloc[:, 197:198], X.iloc[:, 213:214], X.iloc[:, 229:230], 
               X.iloc[:, 245:246], X.iloc[:, 261:262], X.iloc[:, 277:278]], axis=1)

# Step 4: Split data into training and testing sets (80% train, 20% test)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Step 5: Standardize the features (Lasso is sensitive to scale)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert the scaled X_train back to a DataFrame to keep the index alignment with Y_train
X_train_scaled = pd.DataFrame(X_train_scaled, index=X_train.index, columns=X_train.columns)

# Step 6: Fit Lasso regression with a specific alpha value (regularization strength)
lasso_model = Lasso(alpha=0.1)
lasso_model.fit(X_train_scaled, Y_train)

# Step 7: Calculate residuals on the TRAINING set
Y_train_pred = lasso_model.predict(X_train_scaled)
residuals_train = Y_train - Y_train_pred

# Step 8: Create auxiliary regression dataset with squared terms and interaction terms
def create_white_test_data(X):
    X_df = pd.DataFrame(X)  # Input data already has a constant added if necessary
    
    # Add squared terms
    squared_terms = {f'{col}^2': X_df[col] ** 2 for col in X_df.columns}
    
    # Add interaction terms
    interaction_terms = {f'{col1}*{col2}': X_df[col1] * X_df[col2] 
                         for col1, col2 in combinations(X_df.columns, 2)}
    
    # Combine original, squared, and interaction terms into a single DataFrame
    X_expanded = pd.concat([X_df, pd.DataFrame(squared_terms), pd.DataFrame(interaction_terms)], axis=1)

    return X_expanded

# Step 9: Run the adapted White Test
def run_adapted_white_test(X, residuals):
    # Create the expanded auxiliary regression dataset
    X_auxiliary = create_white_test_data(X)
    
    # Fit the auxiliary regression with squared residuals as the target
    auxiliary_model = sm.OLS(residuals**2, sm.add_constant(X_auxiliary)).fit()
    
    # Calculate the White test statistic
    n = X_auxiliary.shape[0]  # Number of observations
    R_squared = auxiliary_model.rsquared
    white_statistic = n * R_squared  # White test statistic (LM test)

    # Degrees of freedom for chi-square test
    df = X_auxiliary.shape[1]  # Degrees of freedom for the test statistic
    
    # p-value from chi-square distribution using scipy
    p_value = stats.chi2.sf(white_statistic, df)
    
    return white_statistic, p_value, auxiliary_model

# Step 10: Apply the adapted White Test on the training data
white_stat, p_value, auxiliary_model = run_adapted_white_test(X_train_scaled, residuals_train)

# Step 11: Print the results
print(f"White Test Statistic: {white_stat}")
print(f"p-value: {p_value}")


Multicollinearity Tests-

Variance Inflation Factor(VIF)

Condition Index

Variance Inflation Factor and Condition Index for the Day-Ahead Market

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from numpy.linalg import eigvals

# Load the data
date_format = "%d/%m/%Y %H:%M"
dat = pd.read_csv("/home/ciaran/Documents/DAM_VAR_1-3.csv", index_col="DeliveryPeriod", parse_dates=True, date_format=date_format)
dat = dat._get_numeric_data()

# Handle missing or infinite values
dat.replace([np.inf, -np.inf], np.nan, inplace=True)  # Replace infs with NaNs
dat.dropna(inplace=True)  # Drop rows with NaNs

# Separate the dependent and independent variables
Y = dat.iloc[:, 0:1]  # The first column as the dependent variable
X = dat.iloc[:, 24:]  # Select all independent variables from the 24th column onward
X = pd.concat([X.iloc[:, :1], X.iloc[:, 144:145], X.iloc[:, 312:313]], axis=1)  # Select specific columns

# Add a constant to the independent variables for the regression
X = sm.add_constant(X)

# Calculate Variance Inflation Factor (VIF) for each selected independent variable
vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print("Variance Inflation Factor (VIF) results:")
print(vif_data)

# Scale the independent variables for Condition Index calculation (excluding the constant)
X_scaled = X.copy()
X_scaled.iloc[:, 1:] = (X.iloc[:, 1:] - X.iloc[:, 1:].mean()) / X.iloc[:, 1:].std(ddof=0)

# Check for NaN or inf values in the scaled data
if np.any(np.isnan(X_scaled)) or np.any(np.isinf(X_scaled)):
    print("Warning: Scaled data contains NaN or inf values.")
    print("\nRows with NaN or inf values in X_scaled:")
    print(X_scaled[np.isnan(X_scaled).any(axis=1) | np.isinf(X_scaled).any(axis=1)])
else:
    # Compute the eigenvalues of the scaled independent variable matrix
    eigenvalues = eigvals(np.dot(X_scaled.iloc[:, 1:].T, X_scaled.iloc[:, 1:]))

    # Calculate the Condition Index for each eigenvalue
    condition_index = np.sqrt(eigenvalues.max() / eigenvalues)

    print("\nCondition Index results:")
    for i, ci in enumerate(condition_index):
        print(f"Condition Index {i+1}: {ci:.2f}")

# Interpretation:
# - High VIF (> 10) indicates significant multicollinearity.
# - Condition Index above 30 suggests severe multicollinearity.


Variance Inflation Factor and Condition Index for the Balancing Market

In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import datetime as dt
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from numpy.linalg import eigvals

# Define date format for parsing
date_format = "%m/%d/%Y %H:%M"

# Function to parse dates
date_format = lambda date: dt.datetime.strptime(date, date_format)

# Read CSV without frequency information and parse dates
dat = pd.read_csv("/home/ciaran/Documents/BM_data.csv", index_col="SettlementPeriod", parse_dates=True, date_format=date_format)

# Specify targets for forecasting
dat = dat.drop(["index"], axis=1)
dat = pd.DataFrame(dat)
dat = dat.bfill(axis='rows')
dat = dat.ffill(axis='rows')
dat = dat._get_numeric_data()

# Select the relevant time series for analysis
Y = dat.iloc[:, 0:1]  # EURPrices column
X = dat.iloc[:, 16:]  # Other variables
X = pd.concat([X.iloc[:, :1], X.iloc[:, 49:50], X.iloc[:, 98:99], X.iloc[:, 147:148], X.iloc[:, 148:149],
X.iloc[:, 149:150], X.iloc[:, 197:198], X.iloc[:, 213:214], X.iloc[:, 229:230], 
X.iloc[:, 245:246], X.iloc[:, 261:262], X.iloc[:, 277:278]], axis=1)

# Add a constant to the independent variables for the regression
X = sm.add_constant(X)

# Calculate Variance Inflation Factor (VIF) for each selected independent variable
vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print("Variance Inflation Factor (VIF) results:")
print(vif_data)

# Scale the independent variables for Condition Index calculation (excluding the constant)
X_scaled = X.copy()
X_scaled.iloc[:, 1:] = (X.iloc[:, 1:] - X.iloc[:, 1:].mean()) / X.iloc[:, 1:].std(ddof=0)

# Check for NaN or inf values in the scaled data
if np.any(np.isnan(X_scaled)) or np.any(np.isinf(X_scaled)):
    print("Warning: Scaled data contains NaN or inf values.")
    print("\nRows with NaN or inf values in X_scaled:")
    print(X_scaled[np.isnan(X_scaled).any(axis=1) | np.isinf(X_scaled).any(axis=1)])
else:
    # Compute the eigenvalues of the scaled independent variable matrix
    eigenvalues = eigvals(np.dot(X_scaled.iloc[:, 1:].T, X_scaled.iloc[:, 1:]))

    # Calculate the Condition Index for each eigenvalue
    condition_index = np.sqrt(eigenvalues.max() / eigenvalues)

    print("\nCondition Index results:")
    for i, ci in enumerate(condition_index):
        print(f"Condition Index {i+1}: {ci:.2f}")


Correlation Analysis-

Pearson Correlation Coefficient

Spearman Rank Correlation

Pearson Correlation Coefficient for the Day-Ahead Market

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load the data
date_format = "%d/%m/%Y %H:%M"
dat = pd.read_csv("/home/ciaran/Documents/DAM_VAR_1-3.csv", index_col="DeliveryPeriod", parse_dates=True, date_format=date_format)
dat = dat._get_numeric_data()

# Handle missing or infinite values
dat.replace([np.inf, -np.inf], np.nan, inplace=True)  # Replace infs with NaNs
dat.dropna(inplace=True)  # Drop rows with NaNs

# Select dependent and independent variables
Y = dat.iloc[:, 0:1]  # The first column as the dependent variable
X = dat.iloc[:, 24:]  # Select all independent variables from the 24th column onward
X = pd.concat([X.iloc[:, 144:145], X.iloc[:, 312:313]], axis=1)  # Select specific columns

# Rename columns for better readability
Y.columns = ['DAM Prices']  # Rename dependent variable
X.columns = ['Wind Forecast', 'Demand Forecast']  # Rename independent variables

# Combine Y and X for correlation analysis
combined_data = pd.concat([Y, X], axis=1)

# Calculate the Pearson correlation matrix
correlation_matrix = combined_data.corr(method='pearson')

# Display the correlation matrix
print("Pearson Correlation Matrix:")
print(correlation_matrix)

# Create the heatmap plot
plt.figure(figsize=(12, 10))  # Increase figure size for clarity
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', vmin=-1, vmax=1,
            xticklabels=combined_data.columns, yticklabels=combined_data.columns, cbar_kws={"shrink": .75})  # Add color bar
plt.title("Pearson Correlation Heatmap: Day-Ahead Market vs Forecast Variables", fontsize=16, fontweight='bold', pad=20)  # More informative title
plt.xticks(fontsize=12, rotation=45, ha='right')  # Rotate and adjust x-axis labels for better readability
plt.yticks(fontsize=12)  # Adjust y-axis labels font size
plt.xlabel('Variables', fontsize=14, labelpad=15)  # Custom x-axis label with padding
plt.ylabel('Variables', fontsize=14, labelpad=15)  # Custom y-axis label with padding

# Show the plot
plt.tight_layout()  # Ensures that labels and titles don't overlap with the figure
plt.savefig('/home/ciaran/Pearson_Correlation_DAM.jpg', format='jpg', dpi=500)

plt.show()


Pearson Correlation Coefficient for the Balancing Market

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt

# Define date format for parsing
date_format = "%m/%d/%Y %H:%M"

# Function to parse dates
date_format = lambda date: dt.datetime.strptime(date, date_format)

# Read CSV without frequency information and parse dates
dat = pd.read_csv("/home/ciaran/Documents/BM_data.csv", index_col="SettlementPeriod", parse_dates=True, date_format=date_format)

# Handle missing or infinite values
dat = dat.drop(["index"], axis=1)  # Remove unnecessary 'index' column
dat = pd.DataFrame(dat)
dat = dat.bfill(axis='rows')  # Backward fill to handle missing values
dat = dat.ffill(axis='rows')  # Forward fill to handle missing values
dat = dat._get_numeric_data()  # Only keep numeric data

# Select relevant time series for analysis
Y = dat.iloc[:, 0:1]  # BM Prices as the dependent variable (first column)
X = dat.iloc[:, 16:]  # Other independent variables (from 17th column onward)

# Select specific columns for X and rename them for clarity
X = pd.concat([X.iloc[:, :1], X.iloc[:, 49:50], X.iloc[:, 98:99], X.iloc[:, 147:148], X.iloc[:, 148:149],
               X.iloc[:, 149:150], X.iloc[:, 197:198], X.iloc[:, 213:214], X.iloc[:, 229:230], 
               X.iloc[:, 245:246], X.iloc[:, 261:262], X.iloc[:, 277:278]], axis=1)

# Rename columns for better understanding
Y.columns = ['BM Prices']  # Dependent variable label
X.columns = ['Past BM Prices', 'BM Volume', 'Wind Difference', 'Carbon Price', 'Gas Price',
             'Past DAM Prices', 'Physical Notifications Volume', 'Net Interconnector Schedule', 
             'Renewable Forecast', 'Indep_Var_10', 'Demand Forecast', 'Future DAM Prices']  # Independent variable labels

# Combine Y and X for correlation analysis
combined_data = pd.concat([Y, X], axis=1)

# Calculate the Pearson correlation matrix
correlation_matrix = combined_data.corr(method='pearson')

# Display the correlation matrix in the console
print("Pearson Correlation Matrix:")
print(correlation_matrix)

# Create the heatmap plot
plt.figure(figsize=(14, 12))  # Increase figure size for clarity
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', vmin=-1, vmax=1,
            xticklabels=combined_data.columns, yticklabels=combined_data.columns, cbar_kws={"shrink": .75})  # Add color bar
plt.title("Pearson Correlation Heatmap: Balancing Market vs Forecast Variables", fontsize=16, fontweight='bold', pad=20)  # Improved title
plt.xticks(fontsize=12, rotation=45, ha='right')  # Rotate x-axis labels for better readability
plt.yticks(fontsize=12)  # Adjust font size of y-axis labels
plt.xlabel('Variables', fontsize=14, labelpad=15)  # Custom x-axis label with padding
plt.ylabel('Variables', fontsize=14, labelpad=15)  # Custom y-axis label with padding

# Show the plot with tight layout
plt.tight_layout()  # Ensure no overlap of labels or titles
plt.savefig('/home/ciaran/Pearson_Correlation_BM.jpg', format='jpg', dpi=500)

plt.show()


Spearman Rank Correlation for the Day-Ahead Market

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import spearmanr

# Load the data
date_format = "%d/%m/%Y %H:%M"
dat = pd.read_csv("/home/ciaran/Documents/DAM_VAR_1-3.csv", index_col="DeliveryPeriod", parse_dates=True, date_format=date_format)
dat = dat._get_numeric_data()

# Handle missing or infinite values
dat.replace([np.inf, -np.inf], np.nan, inplace=True)  # Replace infs with NaNs
dat.dropna(inplace=True)  # Drop rows with NaNs

Y = dat.iloc[:, 0:1]  
X = dat.iloc[:, 24:]  
X = pd.concat([X.iloc[:, 144:145], X.iloc[:, 312:313]], axis=1)  

# Rename columns for better readability
Y.columns = ['DAM Prices']  # Rename dependent variable
X.columns = ['Wind Forecast', 'Demand Forecast']  # Rename independent variables

# Combine Y and X for correlation analysis
combined_data = pd.concat([Y, X], axis=1)

# Initialize an empty DataFrame to store Spearman correlation coefficients
spearman_corr_matrix = pd.DataFrame(index=combined_data.columns, columns=combined_data.columns)

# Calculate Spearman correlation for each pair of variables
for col1 in combined_data.columns:
    for col2 in combined_data.columns:
        corr, _ = spearmanr(combined_data[col1], combined_data[col2])
        spearman_corr_matrix.loc[col1, col2] = corr

# Convert correlation values to numeric type
spearman_corr_matrix = spearman_corr_matrix.astype(float)

# Display the Spearman correlation matrix
print("Spearman Correlation Matrix:")
print(spearman_corr_matrix)

# Create the heatmap plot for Spearman Rank Correlation
plt.figure(figsize=(12, 10))  # Increase figure size for clarity
sns.heatmap(spearman_corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', vmin=-1, vmax=1,
            xticklabels=combined_data.columns, yticklabels=combined_data.columns, cbar_kws={"shrink": .75})  # Add color bar
plt.title("Spearman Rank Correlation Heatmap: Day-Ahead Market vs Forecast Variables", fontsize=16, fontweight='bold', pad=20)  # More informative title
plt.xticks(fontsize=12, rotation=45, ha='right')  # Rotate and adjust x-axis labels for better readability
plt.yticks(fontsize=12)  # Adjust y-axis labels font size
plt.xlabel('Variables', fontsize=14, labelpad=15)  # Custom x-axis label with padding
plt.ylabel('Variables', fontsize=14, labelpad=15)  # Custom y-axis label with padding

# Show the plot
plt.tight_layout()  # Ensures that labels and titles don't overlap with the figure
plt.savefig('/home/ciaran/Spearman_Correlation_DAM.jpg', format='jpg', dpi=500)

plt.show()


Spearman Rank Correlation for the Balancing Market

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt

# Define date format for parsing
date_format = "%m/%d/%Y %H:%M"

# Function to parse dates
date_format = lambda date: dt.datetime.strptime(date, date_format)

# Read CSV without frequency information and parse dates
dat = pd.read_csv("/home/ciaran/Documents/BM_data.csv", index_col="SettlementPeriod", parse_dates=True, date_format=date_format)

# Handle missing or infinite values
dat = dat.drop(["index"], axis=1)  # Remove unnecessary 'index' column
dat = pd.DataFrame(dat)
dat = dat.bfill(axis='rows')  # Backward fill to handle missing values
dat = dat.ffill(axis='rows')  # Forward fill to handle missing values
dat = dat._get_numeric_data()  # Only keep numeric data

# Select relevant time series for analysis
Y = dat.iloc[:, 0:1]  # BM Prices as the dependent variable (first column)
X = dat.iloc[:, 16:]  # Other independent variables (from 17th column onward)

# Select specific columns for X and rename them for clarity
X = pd.concat([X.iloc[:, :1], X.iloc[:, 49:50], X.iloc[:, 98:99], X.iloc[:, 147:148], X.iloc[:, 148:149],
               X.iloc[:, 149:150], X.iloc[:, 197:198], X.iloc[:, 213:214], X.iloc[:, 229:230], 
               X.iloc[:, 245:246], X.iloc[:, 261:262], X.iloc[:, 277:278]], axis=1)

# Rename columns for better understanding
Y.columns = ['BM Prices']  # Dependent variable label
X.columns = ['Past BM Prices', 'BM Volume', 'Wind Difference', 'Carbon Price', 'Gas Price',
             'Past DAM Prices', 'Physical Notifications Volume', 'Net Interconnector Schedule', 
             'Renewable Forecast', 'Indep_Var_10', 'Demand Forecast', 'Future DAM Prices']  # Independent variable labels

# Combine Y and X for correlation analysis
combined_data = pd.concat([Y, X], axis=1)

# Calculate the Spearman correlation matrix
correlation_matrix = combined_data.corr(method='spearman')

# Display the correlation matrix in the console
print("Spearman Rank Correlation Matrix:")
print(correlation_matrix)

# Create the heatmap plot
plt.figure(figsize=(14, 12))  # Increase figure size for clarity
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', vmin=-1, vmax=1,
            xticklabels=combined_data.columns, yticklabels=combined_data.columns, cbar_kws={"shrink": .75})  # Add color bar
plt.title("Spearman Rank Correlation Heatmap: Balancing Market vs Forecast Variables", fontsize=16, fontweight='bold', pad=20)  # Improved title
plt.xticks(fontsize=12, rotation=45, ha='right')  # Rotate x-axis labels for better readability
plt.yticks(fontsize=12)  # Adjust font size of y-axis labels
plt.xlabel('Variables', fontsize=14, labelpad=15)  # Custom x-axis label with padding
plt.ylabel('Variables', fontsize=14, labelpad=15)  # Custom y-axis label with padding

# Show the plot with tight layout
plt.tight_layout()  # Ensure no overlap of labels or titles
plt.savefig('/home/ciaran/Spearman_Correlation_BM.jpg', format='jpg', dpi=500)

plt.show()


Decomposition-

Seasonal and Trend Decomposition using LOESS (STL Decomposition)



STL Decomposition for the DAM

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import STL

# Load the data
date_format = "%d/%m/%Y %H:%M"
dat = pd.read_csv("/home/ciaran/Documents/DAM_VAR_1-3.csv", index_col="DeliveryPeriod", parse_dates=True, date_format=date_format)
dat = dat._get_numeric_data()

# Handle missing or infinite values (replace infinite values, fill NaNs instead of dropping)
dat.replace([np.inf, -np.inf], np.nan, inplace=True)
dat.fillna(method='ffill', inplace=True)  # Forward fill to handle NaNs

# Select the time series for decomposition (e.g., DAM prices)
Y = dat.iloc[:, 0]  # Assuming the first column contains the DAM prices

# Apply STL decomposition (adjust the seasonal parameter based on data frequency)
seasonal_period = 23  # e.g., if data is hourly and you expect daily seasonality, use 24
stl = STL(Y, seasonal=seasonal_period)  # Adjust seasonal as necessary

# Fit the decomposition model
result = stl.fit()

# Extract the components
trend = result.trend
seasonal = result.seasonal
resid = result.resid

# Improved plot settings for publication
plt.figure(figsize=(12, 10), dpi=500)  # High DPI for better resolution

# Define a consistent color palette
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']  # Blue, orange, green, red

# Plot the original time series
plt.subplot(411)
plt.plot(Y, label='Original', color=colors[0], linewidth=1.5)
plt.title('Day-Ahead Market Prices', fontsize=14, weight='bold')
plt.ylabel('Price (EUR)', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend(loc='best', fontsize=10)

# Plot the trend component
plt.subplot(412)
plt.plot(trend, label='Trend', color=colors[1], linewidth=1.5)
plt.title('Trend Component', fontsize=14, weight='bold')
plt.ylabel('Trend', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend(loc='best', fontsize=10)

# Plot the seasonal component
plt.subplot(413)
plt.plot(seasonal, label='Seasonal', color=colors[2], linewidth=1.5)
plt.title('Seasonal Component', fontsize=14, weight='bold')
plt.ylabel('Seasonality', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend(loc='best', fontsize=10)

# Plot the residual component
plt.subplot(414)
plt.plot(resid, label='Residual', color=colors[3], linewidth=1.5)
plt.title('Residual Component', fontsize=14, weight='bold')
plt.ylabel('Residual', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend(loc='best', fontsize=10)

# Final layout adjustments
plt.tight_layout(pad=2.0)
plt.xlabel('Date', fontsize=12)

# Save the figure as a high-quality PNG file
plt.savefig('/home/ciaran/stl_decomposition_plot.jpg', dpi=500, bbox_inches='tight')
plt.show()


STL Decomposition for the BM

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import STL

# Load the data (adjust the date format and path accordingly)
date_format = "%m/%d/%Y %H:%M"
dat = pd.read_csv("/home/ciaran/Documents/BM_data.csv", index_col="SettlementPeriod", parse_dates=True, date_format=date_format)

# Drop unnecessary columns, fill missing values, and ensure only numeric data
dat = dat.drop(["index"], axis=1)
dat = pd.DataFrame(dat)
dat = dat.bfill(axis='rows')
dat = dat.ffill(axis='rows')
dat = dat._get_numeric_data()

# Select the time series for decomposition (assuming the first column contains the BM prices)
Y = dat.iloc[:, 0]  # Adjust column index if necessary

period = 48

# Apply STL decomposition with the specified period
stl = STL(Y, seasonal=13, period=period)

# Fit the decomposition model
result = stl.fit()

# Extract the components
trend = result.trend
seasonal = result.seasonal
resid = result.resid

# Improved plot settings for publication
plt.figure(figsize=(12, 10), dpi=500)  # High DPI for better resolution

# Define a consistent color palette
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']  # Blue, orange, green, red

# Plot the original time series
plt.subplot(411)
plt.plot(Y, label='Original', color=colors[0], linewidth=1.5)
plt.title('Balancing Market Prices', fontsize=14, weight='bold')
plt.ylabel('Price (EUR)', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend(loc='best', fontsize=10)

# Plot the trend component
plt.subplot(412)
plt.plot(trend, label='Trend', color=colors[1], linewidth=1.5)
plt.title('Trend Component', fontsize=14, weight='bold')
plt.ylabel('Trend', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend(loc='best', fontsize=10)

# Plot the seasonal component
plt.subplot(413)
plt.plot(seasonal, label='Seasonal', color=colors[2], linewidth=1.5)
plt.title('Seasonal Component', fontsize=14, weight='bold')
plt.ylabel('Seasonality', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend(loc='best', fontsize=10)

# Plot the residual component
plt.subplot(414)
plt.plot(resid, label='Residual', color=colors[3], linewidth=1.5)
plt.title('Residual Component', fontsize=14, weight='bold')
plt.ylabel('Residual', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend(loc='best', fontsize=10)

# Final layout adjustments
plt.tight_layout(pad=2.0)
plt.xlabel('Date', fontsize=12)

# Save the figure as a high-quality PNG file
plt.savefig('/home/ciaran/bm_stl_decomposition_plot.jpg', dpi=500, bbox_inches='tight')
plt.show()


Residual Analysis-

Residual Plots

Autocorrelation in Residuals


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load your data
dat1 = pd.read_csv("/home/ciaran/QRA&Q-Ave(QR&CP)/DAM_QRA/QR/LEAR_Q_DAM_1-12.csv")

# Extract real prices and predictions
real_prices = dat1[['EURPrices+{}'.format(i) for i in range(0, 23)]].dropna().stack().reset_index(drop=True)
predictions = dat1[['EURPrices+{}_Forecast_50'.format(i) for i in range(0, 23)]].dropna().stack().reset_index(drop=True)

# Calculate residuals
residuals = real_prices - predictions
# Create a DataFrame for easier plotting
df_residuals = pd.DataFrame({
    'Fitted Values': predictions,
    'Residuals': residuals
})

plt.figure(figsize=(10, 6))

# Highlight outliers
std_residuals = df_residuals['Residuals'].std()
outliers = abs(df_residuals['Residuals']) > 2 * std_residuals

sns.scatterplot(x='Fitted Values', y='Residuals', data=df_residuals, hue=outliers, 
                palette={False: 'blue', True: 'orange'}, alpha=0.6)

# Add LOWESS smoothing line
sns.regplot(x='Fitted Values', y='Residuals', data=df_residuals, lowess=True, 
            line_kws={'color': 'red', 'lw': 1}, scatter=False)

plt.axhline(0, color='black', lw=1, ls='--')
plt.ylim(-100, 100)  # Adjust y-axis limits
plt.title('Residual Plot of Predicted vs Actual Prices (DAM/BM)')
plt.xlabel('Fitted Values (Predicted Prices in EUR)')
plt.ylabel('Residuals (Actual - Predicted Prices in EUR)')
plt.grid(True)
plt.show()


In [ ]:
dat1 = pd.read_csv("/home/ciaran/QRA&Q-Ave(QR&CP)/DAM_QRA/QR/lgbm_Q_DAM_1-12.csv")

# Extract real prices and predictions
real_prices = dat1[['EURPrices+{}'.format(i) for i in range(0, 23)]].dropna().stack().reset_index(drop=True)
predictions = dat1[['EURPrices+{}_Forecast_50'.format(i) for i in range(0, 23)]].dropna().stack().reset_index(drop=True)

# Calculate residuals
residuals = real_prices - predictions
# Create a DataFrame for easier plotting
df_residuals = pd.DataFrame({
    'Fitted Values': predictions,
    'Residuals': residuals
})

plt.figure(figsize=(10, 6))

# Highlight outliers
std_residuals = df_residuals['Residuals'].std()
outliers = abs(df_residuals['Residuals']) > 2 * std_residuals

sns.scatterplot(x='Fitted Values', y='Residuals', data=df_residuals, hue=outliers, 
                palette={False: 'blue', True: 'orange'}, alpha=0.6)

# Add LOWESS smoothing line
sns.regplot(x='Fitted Values', y='Residuals', data=df_residuals, lowess=True, 
            line_kws={'color': 'red', 'lw': 1}, scatter=False)

plt.axhline(0, color='black', lw=1, ls='--')
plt.ylim(-100, 100)  # Adjust y-axis limits
plt.title('Residual Plot of Predicted vs Actual Prices (DAM/BM)')
plt.xlabel('Fitted Values (Predicted Prices in EUR)')
plt.ylabel('Residuals (Actual - Predicted Prices in EUR)')
plt.grid(True)
plt.show()

In [ ]:
dat1 = pd.read_csv("/home/ciaran/QRA&Q-Ave(QR&CP)/DAM_QRA/QR/rf_Q_DAM_1-12.csv")

# Extract real prices and predictions
real_prices = dat1[['EURPrices+{}'.format(i) for i in range(0, 23)]].dropna().stack().reset_index(drop=True)
predictions = dat1[['EURPrices+{}_Forecast_50'.format(i) for i in range(0, 23)]].dropna().stack().reset_index(drop=True)

# Calculate residuals
residuals = real_prices - predictions
# Create a DataFrame for easier plotting
df_residuals = pd.DataFrame({
    'Fitted Values': predictions,
    'Residuals': residuals
})

plt.figure(figsize=(10, 6))

# Highlight outliers
std_residuals = df_residuals['Residuals'].std()
outliers = abs(df_residuals['Residuals']) > 2 * std_residuals

sns.scatterplot(x='Fitted Values', y='Residuals', data=df_residuals, hue=outliers, 
                palette={False: 'blue', True: 'orange'}, alpha=0.6)

# Add LOWESS smoothing line
sns.regplot(x='Fitted Values', y='Residuals', data=df_residuals, lowess=True, 
            line_kws={'color': 'red', 'lw': 1}, scatter=False)

plt.axhline(0, color='black', lw=1, ls='--')
plt.ylim(-100, 100)  # Adjust y-axis limits
plt.title('Residual Plot of Predicted vs Actual Prices (DAM/BM)')
plt.xlabel('Fitted Values (Predicted Prices in EUR)')
plt.ylabel('Residuals (Actual - Predicted Prices in EUR)')
plt.grid(True)
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load your data
dat1 = pd.read_csv("/home/ciaran/QRA&Q-Ave(QR&CP)/BM_QRA/QR/LEAR_Q_1-12.csv")
dat1 = pd.DataFrame(dat1)

# Extract real prices and predictions for the BM market
real_prices_BM = dat1[['lag_{}y'.format(i) for i in range(2, 18)]].dropna().stack().reset_index(drop=True)
predictions_BM = dat1[['lag_{}y_Forecast_50'.format(i) for i in range(2, 18)]].dropna().stack().reset_index(drop=True)

# Calculate residuals for the BM market
residuals_BM = real_prices_BM - predictions_BM

# Create a DataFrame for easier plotting
df_residuals_BM = pd.DataFrame({
    'Fitted Values': predictions_BM,
    'Residuals': residuals_BM
})

plt.figure(figsize=(10, 6))

# Highlight outliers
std_residuals_BM = df_residuals_BM['Residuals'].std()
outliers_BM = abs(df_residuals_BM['Residuals']) > 2 * std_residuals_BM

sns.scatterplot(x='Fitted Values', y='Residuals', data=df_residuals_BM, hue=outliers_BM, 
                palette={False: 'blue', True: 'orange'}, alpha=0.6)

# Add LOWESS smoothing line
sns.regplot(x='Fitted Values', y='Residuals', data=df_residuals_BM, lowess=True, 
            line_kws={'color': 'red', 'lw': 1}, scatter=False)

plt.axhline(0, color='black', lw=1, ls='--')
plt.ylim(-500, 500)  # Adjust y-axis limits as necessary
plt.title('Residual Plot of Predicted vs Actual Prices (BM)')
plt.xlabel('Fitted Values (Predicted Prices in EUR)')
plt.ylabel('Residuals (Actual - Predicted Prices in EUR)')
plt.grid(True)
plt.show()


In [ ]:
dat1 = pd.read_csv("/home/ciaran/QRA&Q-Ave(QR&CP)/BM_QRA/QR/lgbm_Q_1-12.csv")
dat1 = pd.DataFrame(dat1)

# Extract real prices and predictions for the BM market
real_prices_BM = dat1[['lag_{}y'.format(i) for i in range(2, 18)]].dropna().stack().reset_index(drop=True)
predictions_BM = dat1[['lag_{}y_Forecast_50'.format(i) for i in range(2, 18)]].dropna().stack().reset_index(drop=True)

# Calculate residuals for the BM market
residuals_BM = real_prices_BM - predictions_BM

# Create a DataFrame for easier plotting
df_residuals_BM = pd.DataFrame({
    'Fitted Values': predictions_BM,
    'Residuals': residuals_BM
})

plt.figure(figsize=(10, 6))

# Highlight outliers
std_residuals_BM = df_residuals_BM['Residuals'].std()
outliers_BM = abs(df_residuals_BM['Residuals']) > 2 * std_residuals_BM

sns.scatterplot(x='Fitted Values', y='Residuals', data=df_residuals_BM, hue=outliers_BM, 
                palette={False: 'blue', True: 'orange'}, alpha=0.6)

# Add LOWESS smoothing line
sns.regplot(x='Fitted Values', y='Residuals', data=df_residuals_BM, lowess=True, 
            line_kws={'color': 'red', 'lw': 1}, scatter=False)

plt.axhline(0, color='black', lw=1, ls='--')
plt.ylim(-500, 500)  # Adjust y-axis limits as necessary
plt.title('Residual Plot of Predicted vs Actual Prices (BM)')
plt.xlabel('Fitted Values (Predicted Prices in EUR)')
plt.ylabel('Residuals (Actual - Predicted Prices in EUR)')
plt.grid(True)
plt.show()


In [ ]:
dat1 = pd.read_csv("/home/ciaran/QRA&Q-Ave(QR&CP)/BM_QRA/QR/rf_Q_1-12.csv")
dat1=pd.DataFrame(dat1)

# Extract real prices and predictions for the BM market
real_prices_BM = dat1[['lag_{}y'.format(i) for i in range(2, 18)]].dropna().stack().reset_index(drop=True)
predictions_BM = dat1[['lag_{}y_Forecast_50'.format(i) for i in range(2, 18)]].dropna().stack().reset_index(drop=True)

# Calculate residuals for the BM market
residuals_BM = real_prices_BM - predictions_BM

# Create a DataFrame for easier plotting
df_residuals_BM = pd.DataFrame({
    'Fitted Values': predictions_BM,
    'Residuals': residuals_BM
})

plt.figure(figsize=(10, 6))

# Highlight outliers
std_residuals_BM = df_residuals_BM['Residuals'].std()
outliers_BM = abs(df_residuals_BM['Residuals']) > 2 * std_residuals_BM

sns.scatterplot(x='Fitted Values', y='Residuals', data=df_residuals_BM, hue=outliers_BM, 
                palette={False: 'blue', True: 'orange'}, alpha=0.6)

# Add LOWESS smoothing line
sns.regplot(x='Fitted Values', y='Residuals', data=df_residuals_BM, lowess=True, 
            line_kws={'color': 'red', 'lw': 1}, scatter=False)

plt.axhline(0, color='black', lw=1, ls='--')
plt.ylim(-500, 500)  # Adjust y-axis limits as necessary
plt.title('Residual Plot of Predicted vs Actual Prices (BM)')
plt.xlabel('Fitted Values (Predicted Prices in EUR)')
plt.ylabel('Residuals (Actual - Predicted Prices in EUR)')
plt.grid(True)
plt.show()


Autocorrelation in Residuals


In [ ]:
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.stats.diagnostic import acorr_ljungbox
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

# Load your data
dat1 = pd.read_csv("/home/ciaran/QRA&Q-Ave(QR&CP)/DAM_QRA/QR/LEAR_Q_DAM_1-12.csv")

# Extract real prices and predictions
real_prices = dat1[['EURPrices+{}'.format(i) for i in range(0, 23)]].dropna().stack().reset_index(drop=True)
predictions = dat1[['EURPrices+{}_Forecast_50'.format(i) for i in range(0, 23)]].dropna().stack().reset_index(drop=True)

# Calculate residuals
residuals = real_prices - predictions

# Plot the Autocorrelation Function (ACF) of the residuals
plt.figure(figsize=(10, 6))
plot_acf(residuals, lags=40, alpha=0.05)
plt.title('Autocorrelation Function (ACF) of Residuals (DAM)')
plt.xlabel('Lags')
plt.ylabel('Autocorrelation')
plt.grid(True)
plt.show()

# Perform the Ljung-Box test for autocorrelation at multiple lags
ljung_box_results = acorr_ljungbox(residuals, lags=[10, 20, 30, 40], return_df=True)

# Display the Ljung-Box test results
print("Ljung-Box Test Results:")
print(ljung_box_results)

In [ ]:
dat1 = pd.read_csv("/home/ciaran/QRA&Q-Ave(QR&CP)/DAM_QRA/QR/lgbm_Q_DAM_1-12.csv")


# Extract real prices and predictions
real_prices = dat1[['EURPrices+{}'.format(i) for i in range(0, 23)]].dropna().stack().reset_index(drop=True)
predictions = dat1[['EURPrices+{}_Forecast_50'.format(i) for i in range(0, 23)]].dropna().stack().reset_index(drop=True)

# Calculate residuals
residuals = real_prices - predictions

# Plot the Autocorrelation Function (ACF) of the residuals
plt.figure(figsize=(10, 6))
plot_acf(residuals, lags=40, alpha=0.05)
plt.title('Autocorrelation Function (ACF) of Residuals (DAM)')
plt.xlabel('Lags')
plt.ylabel('Autocorrelation')
plt.grid(True)
plt.show()

# Perform the Ljung-Box test for autocorrelation at multiple lags
ljung_box_results = acorr_ljungbox(residuals, lags=[10, 20, 30, 40], return_df=True)

# Display the Ljung-Box test results
print("Ljung-Box Test Results:")
print(ljung_box_results)

In [ ]:
dat1 = pd.read_csv("/home/ciaran/QRA&Q-Ave(QR&CP)/DAM_QRA/QR/rf_Q_DAM_1-12.csv")

# Extract real prices and predictions
real_prices = dat1[['EURPrices+{}'.format(i) for i in range(0, 23)]].dropna().stack().reset_index(drop=True)
predictions = dat1[['EURPrices+{}_Forecast_50'.format(i) for i in range(0, 23)]].dropna().stack().reset_index(drop=True)

# Calculate residuals
residuals = real_prices - predictions

# Plot the Autocorrelation Function (ACF) of the residuals
plt.figure(figsize=(10, 6))
plot_acf(residuals, lags=40, alpha=0.05)
plt.title('Autocorrelation Function (ACF) of Residuals (DAM)')
plt.xlabel('Lags')
plt.ylabel('Autocorrelation')
plt.grid(True)
plt.show()

# Perform the Ljung-Box test for autocorrelation at multiple lags
ljung_box_results = acorr_ljungbox(residuals, lags=[10, 20, 30, 40], return_df=True)

# Display the Ljung-Box test results
print("Ljung-Box Test Results:")
print(ljung_box_results)

In [ ]:
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.stats.diagnostic import acorr_ljungbox
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

# Load your data
dat1 = pd.read_csv("/home/ciaran/QRA&Q-Ave(QR&CP)/BM_QRA/QR/LEAR_Q_1-12.csv")
dat1 = pd.DataFrame(dat1)


# Extract real prices and predictions for the BM market
real_prices_BM = dat1[['lag_{}y'.format(i) for i in range(2, 18)]].dropna().stack().reset_index(drop=True)
predictions_BM = dat1[['lag_{}y_Forecast_50'.format(i) for i in range(2, 18)]].dropna().stack().reset_index(drop=True)

# Calculate residuals for the BM market
residuals = real_prices_BM - predictions_BM

# Plot the Autocorrelation Function (ACF) of the residuals
plt.figure(figsize=(10, 6))
plot_acf(residuals, lags=40, alpha=0.05)
plt.title('Autocorrelation Function (ACF) of Residuals (BM)')
plt.xlabel('Lags')
plt.ylabel('Autocorrelation')
plt.grid(True)
plt.show()

# Perform the Ljung-Box test for autocorrelation at multiple lags
ljung_box_results = acorr_ljungbox(residuals, lags=[10, 20, 30, 40], return_df=True)

# Display the Ljung-Box test results
print("Ljung-Box Test Results:")
print(ljung_box_results)

In [ ]:
dat1 = pd.read_csv("/home/ciaran/QRA&Q-Ave(QR&CP)/BM_QRA/QR/lgbm_Q_1-12.csv")
dat1 = pd.DataFrame(dat1)

# Extract real prices and predictions for the BM market
real_prices_BM = dat1[['lag_{}y'.format(i) for i in range(2, 18)]].dropna().stack().reset_index(drop=True)
predictions_BM = dat1[['lag_{}y_Forecast_50'.format(i) for i in range(2, 18)]].dropna().stack().reset_index(drop=True)

# Calculate residuals for the BM market
residuals = real_prices_BM - predictions_BM

# Plot the Autocorrelation Function (ACF) of the residuals
plt.figure(figsize=(10, 6))
plot_acf(residuals, lags=40, alpha=0.05)
plt.title('Autocorrelation Function (ACF) of Residuals (BM)')
plt.xlabel('Lags')
plt.ylabel('Autocorrelation')
plt.grid(True)
plt.show()

# Perform the Ljung-Box test for autocorrelation at multiple lags
ljung_box_results = acorr_ljungbox(residuals, lags=[10, 20, 30, 40], return_df=True)

# Display the Ljung-Box test results
print("Ljung-Box Test Results:")
print(ljung_box_results)

In [ ]:
dat1 = pd.read_csv("/home/ciaran/QRA&Q-Ave(QR&CP)/BM_QRA/QR/rf_Q_1-12.csv")
dat1=pd.DataFrame(dat1)


# Extract real prices and predictions for the BM market
real_prices_BM = dat1[['lag_{}y'.format(i) for i in range(2, 18)]].dropna().stack().reset_index(drop=True)
predictions_BM = dat1[['lag_{}y_Forecast_50'.format(i) for i in range(2, 18)]].dropna().stack().reset_index(drop=True)

# Calculate residuals for the BM market
residuals = real_prices_BM - predictions_BM

# Plot the Autocorrelation Function (ACF) of the residuals
plt.figure(figsize=(10, 6))
plot_acf(residuals, lags=40, alpha=0.05)
plt.title('Autocorrelation Function (ACF) of Residuals (BM)')
plt.xlabel('Lags')
plt.ylabel('Autocorrelation')
plt.grid(True)
plt.show()

# Perform the Ljung-Box test for autocorrelation at multiple lags
ljung_box_results = acorr_ljungbox(residuals, lags=[10, 20, 30, 40], return_df=True)

# Display the Ljung-Box test results
print("Ljung-Box Test Results:")
print(ljung_box_results)